# Qual o jogador vai fazer o primeiro gol?

&nbsp;&nbsp;&nbsp;&nbsp;Este *notebook* visa a preparar os dados para, futuramente, por meio do modelo preditivo, responder à questão de qual jogador marcará o primeiro gol nas partidas.

### Importação das bibliotecas

In [ ]:
# Importar bibliotecas necessárias
import pandas as pd
import numpy as np
import matplotlib as mp
from scipy import stats
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
!pip install skimpy
import skimpy as sk

- ```import pandas as pd```: Importa a biblioteca Pandas e a renomeia como "pd". O Pandas é usado para manipulação e análise de dados em formato tabular (dataframes).

- ```from sklearn.preprocessing import MinMaxScaler```: Importa o `MinMaxScaler` da biblioteca `sklearn.preprocessing`, que é usado para normalizar os dados. Ele transforma as features de um dataset para um intervalo específico, mantendo a proporção entre os valores originais.

- ```import seaborn as sns```: Importa a biblioteca Seaborn e a renomeia como "sns", a qual é usada para visualização de dados estatísticos, facilitando a criação de gráficos.

- ```import matplotlib.pyplot as plt```: Importa a biblioteca Matplotlib e a renomeia como "plt", a qual é útil para criar gráficos, oferecendo controle detalhado sobre cada aspecto dos gráficos.

- ```from sklearn.preprocessing import LabelEncoder```: Importa o `LabelEncoder` da biblioteca `sklearn.preprocessing`. O `LabelEncoder` é usado para converter variáveis categóricas em valores numéricos.

- ```import numpy as np```: Importa a biblioteca NumPy e a renomeia como "np". O NumPy é uma biblioteca fundamental para computação científica em Python, oferecendo suporte para arrays multidimensionais e uma ampla gama de funções matemáticas, lógicas, manipulação de formas, álgebra linear, entre outras.

- ```import matplotlib as mp```: Importa a biblioteca Matplotlib e a renomeia como "mp". A Matplotlib é uma biblioteca de visualização de dados em Python, utilizada principalmente para criar gráficos 2D, como gráficos de linha, barras, histogramas e outros.

- ```from scipy import stats```: Importa o módulo `stats` da biblioteca SciPy. A SciPy é uma biblioteca que estende a funcionalidade do NumPy com ferramentas adicionais para cálculos matemáticos, científicos e de engenharia e o módulo `stats` oferece uma variedade de funções para estatísticas.

- ```from sklearn.preprocessing import StandardScaler```: Importa o `StandardScaler` da biblioteca `sklearn.preprocessing`. O `StandardScaler` é usado para padronizar as features de um dataset, removendo a média e escalando para a variância unitária.

- ```from sklearn.model_selection import train_test_split```: Importa a função `train_test_split` da biblioteca `sklearn.model_selection`. Essa função é usada para dividir um dataset em conjuntos de treino e teste, facilitando a validação do modelo de machine learning.

- ```from sklearn.ensemble import RandomForestClassifier```: Importa o `RandomForestClassifier` da biblioteca `sklearn.ensemble`. O `RandomForestClassifier` é um algoritmo de machine learning que utiliza múltiplas árvores de decisão para classificar dados.

- ```from sklearn.metrics import accuracy_score```: Importa a função `accuracy_score` da biblioteca `sklearn.metrics`. Essa função é usada para calcular a precisão de um modelo de classificação, que é a proporção de previsões corretas feitas pelo modelo em comparação com o total de previsões.

- `import skimpy as sk`: Importa a biblioteca `skimpy` e a renomeia como `sk`. Essa biblioteca foi projetada para facilitar a análise e visualização de dados.

&nbsp;&nbsp;&nbsp;&nbsp;Nesse trecho do código, ocorre a importação das bibliotecas necessárias para realizar tarefas de análise de dados e pré-processamento.


### Carregamento dos conjuntos de dados

In [ ]:
# Carregar o dataset contendo os dados dos jogadores
df_players = pd.read_csv('brazil-serie-a-players-2024-to-2024-stats (2).csv')
# Carregar o dataset contendo os dados das partidas
df_matches = pd.read_csv('brazil-serie-a-matches-2024-to-2024-stats_edit - brazil-serie-a-matches-2024-to-2024-stats_edit.csv')


&nbsp;&nbsp;&nbsp;&nbsp;Nesse trecho de código, utiliza-se o Pandas para carregar um conjunto de dados (dataset) a partir dos arquivos CSV chamados "brazil-serie-a-players-2024-to-2024-stats (2).csv" e "brazil-serie-a-matches-2024-to-2024-stats_edit - brazil-serie-a-matches-2024-to-2024-stats_edit.csv".
   - `pd.read_csv` é uma função do Pandas que lê um arquivo CSV e o transforma em um objeto DataFrame, que é uma estrutura de dados tabular usada para armazenar e manipular dados.
   - Os nomes "df_players" e "df_matches" são escolhidos como nomes das variáveis que armazenam os DataFrames que contém os dados carregados dos arquivos CSV.

- **"brazil-serie-a-players-2024-to-2024-stats (2).csv"**: nome do arquivo CSV que contém informações sobre os jogadores da série A do Campeonato Brasileiro.

- **"brazil-serie-a-matches-2024-to-2024-stats_edit - brazil-serie-a-matches-2024-to-2024-stats_edit.csv"**: nome do arquivo CSV que contém informações sobre as partidas da série A do Campeonato Brasileiro.

&nbsp;&nbsp;&nbsp;&nbsp;Após a execução desse trecho, o conjunto de dados sobre os jogadores é carregado no DataFrame "df_players" e o conjunto de dados sobre as partidas, no DataFrame "df_matches", permitindo a análise e manipulação das informações contidas nos arquivos CSV.

## Exploração de dados

### Seleção de features

In [ ]:
# Define quais colunas do arquivo de jogadores serão utilizadas
columns_players = ['full_name', 'position', 'Current Club', 'goals_overall', 'goals_per_90_overall', 'min_per_goal_overall',
           'shots_on_target_per_90_overall', 'shot_accuraccy_percentage_overall', 'xg_per_90_overall', 'goals_home',
           'goals_away', 'rank_in_club_top_scorer', 'minutes_played_overall', 'shots_on_target_per_game_overall',
           'average_rating_overall']

&nbsp;&nbsp;&nbsp;&nbsp; Esse trecho de código cria uma lista chamda "columns_players" com os nomes das colunas que foram selecionadas do arquivo sobre os jogadores pelo grupo para responder à questão proposta neste *notebook*. Tais colunas foram selecionadas após a exploração dos dados, pois foram consideradas importantes para definir qual jogador marcará o primeiro gol:
- **`full_name`**: Identifica cada jogador de forma única pelo seu nome, permitindo o rastreamento específico de seu desempenho.

- **`position`**: A posição em que o jogador atua.

- **`Current Club`**: O clube em que o jogador atua.

- **`goals_overall`**: O total de gols marcados ao longo dos dois meses de campeonato de 2024 abrangidos pelos dados.

- **`goals_per_90_overall`**: Esta métrica normaliza o número de gols em relação ao tempo de jogo.

- **`min_per_goal_overall`**: Esta coluna indica quantos minutos, em média, um jogador precisa para marcar um gol.

- **`shots_on_target_per_90_overall`**: Número de chutes na direção do gol a cada 90 minutos de jogo.

- **`shot_accuraccy_percentage_overall`**: Mede a porcentagem de precisão dos chutes do jogador.

- **`xg_per_90_overall`**: Indica a quantidade de Gols Esperados que o jogador marque a cada 90 minutos jogados.

- **`goals_home`**: Quantidade de gols marcados em casa.

- **`goals_away`**: Quantidade de gols marcados fora de casa.

- **`rank_in_club_top_scorer`**: A classificação de um jogador como artilheiro do clube.

- **`minutes_played_overall`**: O tempo total jogado pelo jogador ao longo dos dois meses de campeonato de 2024 abrangidos pelos dados.

- **`shots_on_target_per_game_overall`**: Número de chutes na direção do gol a cada partida.

- **`average_rating_overall`**: A classificação média do jogador em suas performances.

In [ ]:
# Define quais colunas do arquivo de partidas serão utilizadas
columns_matches = ['home_team_name', 'away_team_name', 'home_team_goal_timings', 'away_team_goal_timings']

&nbsp;&nbsp;&nbsp;&nbsp; Esse trecho de código cria uma lista chamda "columns_matches" com os nomes das colunas selecionadas do arquivo sobre as partidas para responder à questão proposta neste *notebook*. Tais colunas foram selecionadas após a exploração dos dados pelo grupo, pois foram consideradas importantes para definir qual jogador marcará o primeiro gol:

- **`home_team_name`**: Esta coluna identifica o nome da equipe que joga em casa.

- **`away_team_name`**: Essa coluna identifica o nome da equipe visitante.

- **`home_team_goal_timings`**: Esta coluna contém os tempos em que a equipe da casa marcou seus gols durante a partida.

- **`away_team_goal_timings`**: Da mesma forma, esta coluna registra os tempos dos gols marcados pela equipe visitante.


### Estatística descritiva das colunas e identificação das colunas numéricas e categóricas

In [ ]:
# Mostra as estatísticas descritivas para as colunas numéricas do df_players
df_players.describe()

In [ ]:
# Mostra as estatísticas descritivas para as colunas numéricas do df_matches
df_matches.describe()

In [ ]:
# Gera um resumo de df_players com skimpy
sk.skim(df_players)

In [ ]:
# Gera um resumo de df_matches com skimpy
sk.skim(df_matches)


&nbsp;&nbsp;&nbsp;&nbsp;O comando `.describe()` é utilizado para gerar estatísticas descritivas dos DataFrame do Pandas chamados `df_players` e `df_matches`. Esse comando retorna tabelas com as estatísticas para todas as colunas numéricas dos DataFrames:

- **count**: Contagem do total de elementos presentes para uma determinada coluna.

- **mean**: Média dos valores da coluna.

- **std**: Desvio padrão dos valores da coluna.

- **min**: Valores mínimos de cada colunas.

- **25%, 50%, 75%**: Quartis de cada coluna.

- **max**: Valores máximos de cada coluna.

&nbsp;&nbsp;&nbsp;&nbsp;Enquanto a função `skim()` de `sk` analisa o DataFrame df e retorna um resumo que inclui:
- Tipos de dados de cada coluna.
- Número de valores únicos.
- Número de valores nulos.
- Estatísticas descritivas (como média, desvio padrão, mínimo, máximo) para colunas numéricas.
- Distribuição de valores para colunas categóricas.

&nbsp;&nbsp;&nbsp;&nbsp;Assim, também é possível concluir quais colunas são numéricas e quais são categóricas por meio da função `skim()`. Para as colunas selecionadas de `df_players`, temos as seguintes estatísticas e classificações para as colunas:

<div align="center">
<sub>Quadro 1 - Estatíca Descritiva das colunas selecionadas de "df_players"</sub>
</div>

<div align="center">

| **Stat** | **goals_overall** | **goals_per_90_overall** | **min_per_goal_overall** | **shots_on_target_per_90_overall** | **shot_accuraccy_percentage_overall** | **xg_per_90_overall** | **goals_home** | **goals_away** | **rank_in_club_top_scorer** | **minutes_played_overall** | **shots_on_target_per_game_overall** | **average_rating_overall** |
|----------|------------------:|-------------------------:|-------------------------:|-----------------------------------:|--------------------------------------:|----------------------:|---------------:|---------------:|----------------------------:|--------------------------:|-----------------------------------:|----------------------------:|
| **count**| 607.0             | 607.0                    | 607.0                    | 495.0                              | 495.0                                 | 495.0                 | 607.0          | 607.0          | 607.0                        | 607.0                        | 495.0                               | 495.0                         |
| **mean** | 0.3542            | 0.1123                   | 76.1252                  | 0.5022                             | 32.1489                                | 0.1769                | 0.1993         | 0.1549         | 11.346                       | 294.3278                      | 0.2450                              | 6.7365                        |
| **std**  | 0.7780            | 0.4304                   | 173.0877                 | 1.0813                             | 34.1347                                | 0.3933                | 0.5279         | 0.4586         | 9.0307                       | 262.1851                      | 0.3223                              | 0.7407                        |
| **min**  | 0.0               | 0.0                      | 0.0                      | 0.0                                | 0.0                                   | 0.0                   | 0.0            | 0.0            | 0.0                          | -1.0                          | 0.0                                | 0.0                           |
| **25%**  | 0.0               | 0.0                      | 0.0                      | 0.0                                | 0.0                                   | 0.0                   | 0.0            | 0.0            | 3.0                          | 44.0                          | 0.0                                | 6.64                          |
| **50%**  | 0.0               | 0.0                      | 0.0                      | 0.2                                | 25.0                                  | 0.06                  | 0.0            | 0.0            | 11.0                         | 251.0                         | 0.14                               | 6.84                          |
| **75%**  | 0.0               | 0.0                      | 0.0                      | 0.645                              | 50.0                                  | 0.2                   | 0.0            | 0.0            | 19.0                         | 488.0                         | 0.38                               | 7.065                         |
| **max**  | 5.0               | 6.43                     | 893.0                    | 15.0                               | 100.0                                 | 4.44                  | 4.0            | 3.0            | 30.0                         | 900.0                         | 2.0                                | 8.18                          |
| **Tipo** | **Numérica**      | **Numérica**             | **Numérica**             | **Numérica**                       | **Numérica**                          | **Numérica**          | **Numérica**   | **Numérica**   | **Numérica**                 | **Numérica**                  | **Numérica**                        | **Numérica**                  |

</div>

<div align="center">
<sup>Fonte: Material produzido pelos autores (2024)</sup>
</div>

&nbsp;&nbsp;&nbsp;&nbsp;Para o `df_matches`, as colunas selecionadas são todas categóricas, portanto, os resumos retornados foram:
- **count**: Contagem do total de elementos presentes para uma determinada variável ou coluna.
- **unique**: Indica o número de valores distintos presentes na coluna.
- **top**: Refere-se ao valor mais frequente (o "valor modal") na coluna.
- **freq**: Representa quantas vezes o valor mais frequente aparece na coluna.

<div align="center">
<sub>Quadro 2 - Estatística das colunas selecionadas de "df_matches"</sub>
</div>

<div align="center">

| **Stat** | **home_team_name** | **away_team_name** | **home_team_goal_timings** | **away_team_goal_timings** |
|----------|-------------------:|-------------------:|----------------------------:|----------------------------:|
| **count**| 380                | 380                | 71                         | 62                         |
| **unique**| 20                 | 20                 | 67                         | 57                         |
| **top**  | Internacional       | Bahia              | 42                         | 70                         |
| **freq** | 19                 | 19                 | 3                          | 2                          |
| **Tipo** | **Categórica**      | **Categórica**     | **Categórica**             | **Categórica**             |


</div>

<div align="center">
<sup>Fonte: Material produzido pelos autores (2024)</sup>
</div>

## Pré-Processamento

In [ ]:
# Remove as linhas cujos jogadores jogaram 0 minuto
df_players_sem_linhas = df_players[df_players['minutes_played_home'] != 0]

&nbsp;&nbsp;&nbsp;&nbsp;O trecho de código acima realiza a remoção das linhas de `df_players` em que a quantidade de minutos jogados pelo jogador ('minutes_played_overall') é igual a zero; dado que, para os tempo iguais a zero, não há dados coletados em jogo sobre os jogadores.

In [ ]:
# Cria um df somente com as colunas selecionadas pelo grupo
df_players_sem_colunas = df_players[columns_players]

&nbsp;&nbsp;&nbsp;&nbsp;O trecho de código define o DataFrame `df_players_sem_colunas` a partir de `df_players_em_linhas`, em que há somente as colunas selecionadas pelo grupo.

In [ ]:
# Relaciona cada posição à média de gols de cada posição
mean_goals_by_position = df_players_sem_colunas.groupby('position')['goals_overall'].mean()
mean_goals_by_position

&nbsp;&nbsp;&nbsp;&nbsp;O trecho de código acima calcula a média de gols marcados por jogadores em diferentes posições.
- **`df_players_sem_colunas.groupby('position')`**: agrupa o DataFrame `df_players_sem_colunas` pela coluna `'position'`, ou seja, os dados são organizados em grupos que correspondem a uma posição específica.
- **`['goals_overall']`**: Após o agrupamento, o código seleciona a coluna `'goals_overall'`, que contém o total de gols marcados por cada jogador.
- **`.mean()`**: Esse método calcula a média dos valores na coluna `'goals_overall'` para cada grupo (ou seja, para cada posição). Isso resulta na média de gols para jogadores em cada posição.
- **`mean_goals_by_position`**: Essa variável armazena uma série do Pandas, cujo índice é a posição do jogador e o valor correspondente, a média de gols dos jogadores nessa posição.
- **`mean_goals_by_position`**: Ao colocar a variável sozinha na linha seguinte, o código instrui o Python a exibir o conteúdo dessa variável. Isso permite visualizar a média de gols por posição:
    - **Defender**: 0.201389
    - **Forward**: 0.783505
    - **Goalkeeper**: 0.000000
    - **Midfielder**: 0.472103

In [ ]:
# Substitui a posição pela média de gols daquela posição
df_players_sem_colunas.loc[:, 'position'] = df_players_sem_colunas['position'].map(mean_goals_by_position)

&nbsp;&nbsp;&nbsp;&nbsp;O código acima substitui os valores categ´ricos da coluna 'position' usando os valores calculados em 'mean_goals_by_position'.
- **`df_players_sem_colunas.loc[:, 'position']`**: O método `.loc` é usado para acessar ou modificar partes específicas do DataFrame. `:` indica que todas as linhas serão selecionadas, e `'position'` indica a coluna que será acessada.
- **`df_players_sem_colunas['position'].map(mean_goals_by_position)`**: Esta parte seleciona a coluna `'position'` do DataFrame, que contém as posições dos jogadores e utiliza o método `map()` para substituir cada valor na coluna `'position'` por um valor numérico correspondente, baseado na série `mean_goals_by_position`.

In [ ]:
# Define df_matches_sem_linhas com as linhas de df_matches cujo status é 'complete'
df_matches_sem_linhas = df_matches[df_matches['status'] == 'complete']

&nbsp;&nbsp;&nbsp;&nbsp;Esse trecho de código cria um DataFrame chamado `df_matches_sem_linhas`, que contém apenas as linhas de `df_matches` onde o valor na coluna `'status'` é `'complete'`; dado que os jogos incompletos não possuem dados úteis para treinamento do modelo preditivo.

- **`df_matches[df_matches['status'] == 'complete']`**: Realiza a filtragem do DataFrame com base na condição `df_matches['status'] == 'complete'`. O DataFrame `df_matches` é filtrado para incluir apenas as linhas onde a condição é `True`, ou seja, onde o valor da coluna `'status'` é `'complete'`.

In [ ]:
# Cria o df_matches_sem_colunas somente com as colunas selecionadas pelo grupo (columns_matches)
df_matches_sem_colunas = df_matches_sem_linhas[columns_matches]

&nbsp;&nbsp;&nbsp;&nbsp;O trecho de código define o DataFrame `df_matches_sem_colunas` a partir de `df_matches_em_linhas`, em que há somente as colunas selecionadas pelo grupo ('columns_matches').

In [ ]:
# Deixa somente o tempo do primeiro gol na coluna 'home_team_goal_timings'
df_matches_sem_colunas.loc[:, 'home_team_goal_timings'] = df_matches_sem_colunas['home_team_goal_timings'].astype(str)
df_matches_sem_colunas.loc[:, 'home_team_goal_timings'] = df_matches_sem_colunas['home_team_goal_timings'].apply(lambda x: x.split(',')[0] if ',' in x else x)
df_matches_sem_colunas.loc[:, 'home_team_goal_timings'] = df_matches_sem_colunas['home_team_goal_timings'].str.replace("'", ".")
df_matches_sem_colunas.loc[:, 'home_team_goal_timings'] = df_matches_sem_colunas['home_team_goal_timings'].astype(float)

# Deixa somente o tempo do primeiro gol na coluna 'away_team_goal_timings'
df_matches_sem_colunas.loc[:, 'away_team_goal_timings'] = df_matches_sem_colunas['away_team_goal_timings'].astype(str)
df_matches_sem_colunas.loc[:, 'away_team_goal_timings'] = df_matches_sem_colunas['away_team_goal_timings'].apply(lambda x: x.split(',')[0] if ',' in x else x)
df_matches_sem_colunas.loc[:, 'away_team_goal_timings'] = df_matches_sem_colunas['away_team_goal_timings'].str.replace("'", ".")
df_matches_sem_colunas.loc[:, 'away_team_goal_timings'] = df_matches_sem_colunas['away_team_goal_timings'].astype(float)


&nbsp;&nbsp;&nbsp;&nbsp;Esse trecho de código manipula duas colunas de `df_matches_sem_colunas`, chamadas `'home_team_goal_timings'` e `'away_team_goal_timings'`, que contêm informações sobre os momentos em que os gols foram marcados, tanto pela equipe da casa quanto pela equipe visitante.
- **`.astype(str)`**: Converte para o tipo de dados `string` para garantir que todas as operações subsequentes funcionem corretamente.
- **`.apply(lambda x: x.split(',')[0] if ',' in x else x)`**: Caso haja múltiplos valores separados por vírgulas, apenas o primeiro valor é mantido, isto é, o momento do primeiro gol.
- **`.str.replace("'", ".")`**: Substitui aspas simples (usadas para indicar minutos) por pontos, para facilitar o processamento posterior.

In [ ]:
# Remove espaços em branco antes e depois dos valores nas colunas
df_players_sem_colunas.loc[:, 'Current Club'] = df_players_sem_colunas['Current Club'].str.strip()
df_matches_sem_colunas.loc[:, 'home_team_name'] = df_matches_sem_colunas['home_team_name'].str.strip()
df_matches_sem_colunas.loc[:, 'away_team_name'] = df_matches_sem_colunas['away_team_name'].str.strip()

# Faz o merge entre os DataFrames df_matches_sem_colunas e df_players_sem_colunas
home_team_players = pd.merge(df_matches_sem_colunas, df_players_sem_colunas, left_on='home_team_name', right_on='Current Club')
away_team_players = pd.merge(df_matches_sem_colunas, df_players_sem_colunas, left_on='away_team_name', right_on='Current Club')
combined_df = pd.concat([home_team_players, away_team_players])


- **`str.strip()`**: As colunas `'Current Club'`, `'home_team_name'` e `'away_team_name'` têm os espaços em branco removidos no início e no final dos valores, o que evita problemas na correspondência de valores ao realizar as operações de merge.
- **`home_team_players = pd.merge(df_matches_sem_colunas, df_players_sem_colunas, left_on='home_team_name', right_on='Current Club')`**: Realiza um merge entre `df_matches_sem_colunas` e `df_players_sem_colunas` com base na coluna `'home_team_name'` do primeiro DataFrame e a coluna `'Current Club'` do segundo.
- **`away_team_players = pd.merge(df_matches_sem_colunas, df_players_sem_colunas, left_on='away_team_name', right_on='Current Club')`**: Realiza um merge, mas agora considerando a coluna `'away_team_name'` do primeiro DataFrame, para adicionar as informações dos jogadores dos times visitantes.
- **`combined_df = pd.concat([home_team_players, away_team_players])`**: O DataFrame `combined_df` é criado ao concatenar os DataFrames `home_team_players` e `away_team_players`. Isso gera um único DataFrame que contém as informações de jogadores para ambos os times para cada partida.

In [ ]:
# Define uma função chamada 'comparar_elementos' que compara dois valores
def comparar_elementos(a, b):
    if a < b:
        return 'home'
    elif a > b:
        return 'away'
    else:
        return 'none'

# Cria uma nova coluna no 'combined_df' chamada 'team_first_goal'
# Para cada linha, aplica a função 'comparar_elementos' aos valores das colunas 'home_team_goal_timings' e 'away_team_goal_timings'.
# O resultado ('home', 'away' ou 'none') indica qual time marcou o primeiro gol.
combined_df['team_first_goal'] = combined_df.apply(lambda row: comparar_elementos(row['home_team_goal_timings'], row['away_team_goal_timings']), axis=1)


- **Função `comparar_elementos(a, b)`**: Essa função compara dois valores, que representam os momentos em que os gols foram marcados pelas equipes da casa e visitante, respectivamente. A função retorna `'home'` se o time da casa marcou primeiro, `'away'` se o time visitante marcou primeiro, ou `'none'` se ambos marcaram ao mesmo tempo ou se não houve gol.
- **Criação da Coluna `team_first_goal`**: Utiliza o método `apply` do Pandas para aplicar a função `comparar_elementos` em cada linha do DataFrame `combined_df` aos valores das colunas `'home_team_goal_timings'` e `'away_team_goal_timings'`. O resultado dessa comparação é armazenado em uma nova coluna chamada `'team_first_goal'`, que indica qual time marcou o primeiro gol em cada partida.

In [ ]:
# Define a função 'possible_first_goal' que verifica se um jogador de pode ter sido responsável pelo primeiro gol da partida
def possible_first_goal (player_goals_home, player_goals_away, team_first_goal, current_club, home_team_name, away_team_name):
    if home_team_name == current_club:
        if team_first_goal == 'home':
            if player_goals_home > 0:
                return True # Retorna True se o jogador pode ter marcado o primeiro gol
            else:
                return False # Retorna False se o jogador não marcou gols
        else:
            return False # Retorna False se o time da casa não marcou o primeiro gol
    elif away_team_name == current_club:
        if team_first_goal == 'away':
            if player_goals_away > 0:
                return True # Retorna True se o jogador pode ter marcado o primeiro gol.
            else:
                return False # Retorna False se o jogador não marcou gols.
        else:
            return False # Retorna False se o time visitante não marcou o primeiro gol
    else:
        return False # Retorna False se o clube do jogador não é nem o time da casa nem o visitante.

# Cria uma nova coluna chamada 'possible_first_goal' e aplica a função 'possible_first_goal' para cada linha
combined_df['possible_first_goal'] = combined_df.apply(lambda row: possible_first_goal(row['goals_home'], row['goals_away'], row['team_first_goal'], row['Current Club'], row['home_team_name'], row['away_team_name']), axis=1)

- **Função `possible_first_goal`**: Essa função avalia se um jogador de um determinado clube poderia ter sido responsável pelo primeiro gol da partida.
   - **Parâmetros da Função**:
     - `player_goals_home`: Número de gols marcados pelo jogador para o time da casa.
     - `player_goals_away`: Número de gols marcados pelo jogador para o time visitante.
     - `team_first_goal`: Indica qual time marcou o primeiro gol (`'home'`, `'away'` ou `'none'`).
     - `current_club`: Clube atual do jogador.
     - `home_team_name`: Nome do time da casa.
     - `away_team_name`: Nome do time visitante.
   - **Lógica da Função**:
     - Verifica se o jogador pertence ao time da casa ou ao visitante.
     - Se o time ao qual o jogador pertence marcou o primeiro gol da partida, a função verifica se o jogador marcou algum gol para esse time.
     - Retorna `True` se o jogador pode ter sido o autor do primeiro gol, e `False` caso contrário.
- **Criação da Coluna `possible_first_goal`**: Utiliza o método `apply` do Pandas para aplicar a função `possible_first_goal` em cada linha do DataFrame `combined_df` e o resultado (`True` ou `False`) é armazenado em uma nova coluna chamada `'possible_first_goal'`.

In [ ]:
#
def change_teams_names (current_club, home_team_name, away_team_name):
    if current_club == home_team_name:
        return away_team_name
    elif current_club == away_team_name:
        return home_team_name

combined_df['opponent'] = combined_df.apply(lambda row: change_teams_names(row['Current Club'], row['home_team_name'], row['away_team_name']), axis=1)

- **Função `change_teams_names`**: Essa função é usada para determinar o adversário de um clube em uma partida.
   - **Parâmetros da Função**:
     - `current_club`: O clube atual do jogador.
     - `home_team_name`: O nome do time da casa na partida.
     - `away_team_name`: O nome do time visitante na partida.
   - **Lógica da Função**:
     - Verifica se o `current_club` é o mesmo que o `home_team_name`. Se for, isso significa que o clube atual do jogador é o time da casa, e o adversário será o time visitante (`away_team_name`).
     - Se o `current_club` for igual ao `away_team_name`, então o adversário é o time da casa (`home_team_name`).

-  **Criação da Coluna `opponent`**:
   - A função `change_teams_names` é aplicada a cada linha do DataFrame `combined_df` para determinar o adversário do clube atual do jogador na partida correspondente.
   - O resultado é armazenado em uma nova coluna chamada `'opponent'`, que contém o nome do time adversário para cada linha do DataFrame.

In [ ]:
# Importa o LabelEncoder da biblioteca sklearn para codificar variáveis categóricas em valores numéricos.
label_encoder = LabelEncoder()
combined_df['current_club_encoded'] = label_encoder.fit_transform(combined_df['Current Club'])
combined_df['opponent_encoded'] = label_encoder.fit_transform(combined_df['opponent'])
combined_df['full_name_encoded'] = label_encoder.fit_transform(combined_df['full_name'])

- O `LabelEncoder` é uma ferramenta da biblioteca `scikit-learn` usada para converter valores categóricos (como nomes de clubes ou jogadores) em valores numéricos.
- Uma instância de `LabelEncoder` é criada e armazenada na variável `label_encoder`.
- **`Current Club`**: A coluna `'Current Club'` é codificada em valores numéricos, onde cada clube recebe um número único. O resultado é armazenado na nova coluna `'current_club_encoded'`.
- **`opponent`**: Da mesma forma, a coluna `'opponent'` é codificada, e os valores numéricos correspondentes são armazenados na coluna `'opponent_encoded'`.
- **`full_name`**: A coluna `'full_name'`, que contém os nomes completos dos jogadores, também é codificada em valores numéricos. O resultado é armazenado na coluna `'full_name_encoded'`.

In [ ]:
# Seleciona quais as colunas estarão no DataFrame após o pré-processamento
df = combined_df[['full_name_encoded', 'current_club_encoded', 'opponent_encoded', 'position', 'goals_overall', 'goals_home', 'goals_away', 'goals_per_90_overall',
                 'min_per_goal_overall', 'shots_on_target_per_90_overall', 'shot_accuraccy_percentage_overall', 'xg_per_90_overall',
                 'rank_in_club_top_scorer', 'minutes_played_overall', 'shots_on_target_per_game_overall', 'average_rating_overall', 'possible_first_goal']]

&nbsp;&nbsp;&nbsp;&nbsp;O trecho de código acima seleciona o conjunto de colunas do DataFrame combined_df e cria um novo DataFrame chamado df.

In [ ]:
# Lista de colunas que serão verificadas para valores ausentes (NaN)
columns_to_check = [
    'shots_on_target_per_90_overall',
    'shot_accuraccy_percentage_overall',
    'xg_per_90_overall',
    'shots_on_target_per_game_overall',
    'average_rating_overall'
]

# Remove linhas do DataFrame 'df' que tenham valores ausentes (NaN) em todas as colunas especificadas na lista 'columns_to_check'.
df = df.dropna(subset=columns_to_check, how='all')


- Foi criada uma lista chamada `columns_to_check`, que contém os nomes de colunas específicas que serão verificadas para a presença de valores ausentes. Na análise exploratória, foi identificado que essas colunas apresentam muitos valores nulos em comum.
- O método `dropna()` é usado para remover linhas do DataFrame `df`, o parâmetro `subset=columns_to_check` indica que a remoção de linhas só deve considerar as colunas listadas em `columns_to_check` e o parâmetro `how='all'` especifica que uma linha será removida apenas se todas as colunas em `columns_to_check` tiverem valores ausentes.

In [ ]:
scaler = MinMaxScaler()
columns_to_normalize = ['minutes_played_overall', 'rank_in_club_top_scorer', 'average_rating_overall']
df.loc[:, columns_to_normalize] = scaler.fit_transform(df.loc[:, columns_to_normalize])

- O `MinMaxScaler` é uma ferramenta da biblioteca `scikit-learn` usada para normalizar os valores das colunas. Ele transforma os dados para um intervalo definido.
- A lista `columns_to_normalize` contém os nomes das colunas que serão normalizadas:
     - **`minutes_played_overall`**: O número de minutos jogados pode variar amplamente entre os jogadores; assim, normalizar essa coluna ajuda a colocar os dados em uma escala comparável com outras variáveis.
     - **`rank_in_club_top_scorer`**: Posição do jogador como artilheiro no clube. Essa variável é ordinal; dessa forma, normalizar ajuda a reduzir o impacto da variação de escala entre esta e outras variáveis.
     - **`average_rating_overall`**: Avaliação média do jogador em todas as competições. Normalizar essa variável ajuda a garantir que todas as variáveis estejam em um intervalo comum para a comparação no modelo.
- O método `fit_transform()` do `MinMaxScaler` é aplicado às colunas especificadas. Ele ajusta os valores de cada coluna para o intervalo de 0 a 1, onde 0 representa o valor mínimo e 1 o valor máximo original da coluna.
- A operação é realizada diretamente nas colunas do DataFrame `df`, substituindo os valores originais pelos valores normalizados.

## Relação entre variáveis escolhidas

In [ ]:
# Visualização da matriz de correlação com um heatmap
correlation_matrix = df.corr()
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Matriz de Correlação')
plt.show()

&nbsp;&nbsp;&nbsp;&nbsp;O método `df.corr()` calcula a correlação de Pearson por padrão entre as colunas numéricas do DataFrame; enquanto `sns.heatmap()` cria o heatmap a partir da matriz de correlação, cujos principais parâmetros são:
- `annot=True`: Exibe os valores de correlação diretamente no gráfico.
- `cmap='coolwarm'`: Escolhe a paleta de cores que vai do azul (negativo) ao vermelho (positivo).
- `vmin` e `vmax`: Definem os limites da escala de cores (de -1 a 1, que são os valores possíveis de correlação).
&nbsp;&nbsp;&nbsp;&nbsp;Assim, o heatmap facilita a visualização de quais colunas estão mais fortemente correlacionadas.Na matriz de correlação, os valores variam de `-1` a `1`:
- **1** indica uma correlação perfeita positiva.
- **-1** indica uma correlação perfeita negativa.
- **0** indica que não há correlação linear.

In [ ]:
# Violin Plot comparando a distribuição de goals_overall e possible_first_goal
plt.figure(figsize=(10, 6))
sns.violinplot(x='goals_overall', y='possible_first_goal', data=df, inner='quartile')
plt.title('Comparação entre goals_overall e possible_first_goal')
plt.show()

&nbsp;&nbsp;&nbsp;&nbsp;O trecho de código acima cria um gráfico do tipo *Violin Plot* para comparar a distribuição da variável `goals_per_90_overall` com a variável `possible_first_goal`:

- `plt.figure(figsize=(10, 6))`: Esse trecho cria uma nova figura onde o gráfico será desenhado.
- `sns.violinplot`: Essa função do Seaborn cria um *Violin Plot*, que mostra tanto a distribuição da variável quanto a sua densidade.
- `y='position_encoded'`: A variável `possible_first_goal` é mapeada no eixo y, representando a possibilidade do jogador ter marcado o primeiro gol em determinada partida.
- `x='goals_per_90_overall'`: A variável numérica `goals_per_90_overall` é mapeada no eixo x, representando a quantidade de gols marcados por 90 minutos jogados.
- `data=df`: O DataFrame `df` contém os dados que serão usados para plotar o gráfico.
- `inner='quartile'`: Adiciona linhas ao *Violin Plot* para indicar os quartis (divisões de 25%) dentro da distribuição dos dados.
- `plt.title('Violin Plot: Goals per 90 Overall by Position')`: Define o título do gráfico.
- `plt.show()`: Mostra o gráfico gerado na célula de código.

&nbsp;&nbsp;&nbsp;&nbsp;Nesse cenário, um *Violin Plot* - representação gráfica que combina características de um boxplot (gráfico de caixa) e de um gráfico de densidade kernel - é utilizado para visualizar a distribuição de um conjunto de dados, permitindo a observação da forma de distribuição, a mediana, os quartis e a densidade dos dados. Cada lado do "violino" representa a distribuição dos dados de forma simétrica em torno de uma linha central, que corresponde ao valor médio ou mediano; enquanto sua largura do indica a densidade dos dados naquele valor de y, assim, áreas mais largas representam maior densidade de dados.

&nbsp;&nbsp;&nbsp;&nbsp;Dessa forma, O gráfico 'Comparação entre goals_overall e possible_first_goal' demonstra que quando nenhum gol é marcado (`goals_overall = 0`), não há variação na métrica `possible_first_goal`; ela é consistentemente zero. Ademais, conforme o número de gols aumenta, os "violinos" ficam mais estreitos e alongados, porém há diferentes padrões:
- Gols = 1 e 2: Há alguma variação nos valores de `possible_first_goal`, mas a distribuição não é muito ampla.
- Gols = 3, 4 e 5: A distribuição passa a ser mais centrada e menos dispersa; logo, a métrica `possible_first_goal` é mais consistente à medida que mais gols são marcados.

&nbsp;&nbsp;&nbsp;&nbsp;Portanto, depreende-se que a métrica `possible_first_goal` tende a se estabilizar e a se concentrar à medida que o número de gols aumenta. Quando nenhum gol é marcado, essa métrica é consistentemente zero, e, em situações com mais gols, a probabilidade associada a marcação do primeiro gol se torna mais previsível ou menos variável.

In [ ]:
# Cria um boxplot para comparar a distribuição de 'goals_per_90_overall' com base em 'possible_first_goal'
plt.figure(figsize=(8, 6))
sns.boxplot(x='possible_first_goal', y='goals_per_90_overall', data=df)
plt.title('Boxplot de Gols por 90 minutos vs Possível Primeiro Gol')
plt.xlabel('possible_first_goal')
plt.ylabel('goals_per_90_overall')
plt.show()

&nbsp;&nbsp;&nbsp;&nbsp;O trecho de código acima gera um boxplot que compara a distribuição dos valores de `goals_per_90_overall` (gols por 90 minutos) para as duas categorias de `possible_first_goal`, "True" e "False".
- `plt.figure(figsize=(8, 6))`: Define o tamanho do gráfico.
- Eixo X (`x='possible_first_goal'`): Cada boxplot será criado para as diferentes categorias da coluna `possible_first_goal`, que indica se o jogador poderia ter sido o responsável pelo primeiro gol.
- Eixo Y (`y='goals_per_90_overall'`): O boxplot mostrará a distribuição dos valores de `goals_per_90_overall` (gols por 90 minutos) para cada categoria de `possible_first_goal`.
- `data=df`: Indica que os dados para o gráfico vêm do DataFrame `df`.
-`plt.title('Boxplot de Gols por 90 minutos vs Possível Primeiro Gol')`: Define o título do gráfico.
- `plt.xlabel('possible_first_goal')`: Define o rótulo do eixo X.
- `plt.ylabel('goals_per_90_overall')`: Define o rótulo do eixo Y.
-`plt.show()`: Exibe o gráfico gerado.

&nbsp;&nbsp;&nbsp;&nbsp;Nesse sentido, o boxplot é uma ferramenta visual útil para entender se existe uma relação entre o número de gols por 90 minutos e a possibilidade de o jogador ter sido o responsável pelo primeiro gol da partida; visto que é possível identificar, no gráfico, os seguintes elementos:
- Mediana: A linha central em cada boxplot, representando o valor mediano de `goals_per_90_overall` para cada categoria de `possible_first_goal`.
- Dispersão dos dados: As caixas mostram o intervalo interquartil, e as linhas estendem-se até os dados dentro de 1.5 vezes o intervalo interquartil.
- Outliers: Valores fora dos whiskers são considerados outliers e aparecem como pontos individuais.


&nbsp;&nbsp;&nbsp;&nbsp;Desse modo, o gráfico compara a distribuição dos gols marcados por 90 minutos (representados pela variável `goals_per_90_overall`) em relação à possibilidade de um jogador marcar o primeiro gol na partida (representado por `possible_first_goal`, com as categorias "False" e "True"). O eixo Y (`goals_per_90_overall`), mostra a quantidade de gols marcados por 90 minutos jogados e o eixo X (`possible_first_goal`) mostra duas categorias:
- `False`: Jogadores que não são considerados como prováveis de marcar o primeiro gol na partida.
- `True`: Jogadores que são considerados como prováveis de marcar o primeiro gol na partida.

&nbsp;&nbsp;&nbsp;&nbsp;Assim, para os jogadores que não são considerados prováveis de marcar o primeiro gol (`False`), a maioria tem um número muito baixo de gols por 90 minutos, com muitos valores em torno de 0. Enquanto que, para os jogadores que são considerados prováveis de marcar o primeiro gol (`True`), a distribuição é mais concentrada em valores mais altos de gols por 90 minutos. A mediana está acima de 0, indicando que esses jogadores, em média, têm um desempenho melhor em termos de gols por 90 minutos. Portanto, jogadores que são classificados como prováveis de marcar o primeiro gol têm, em geral, uma taxa de gols por 90 minutos maior do que aqueles que não são.


In [ ]:
# Separando os dados em features (X) e target (y)
y = df['possible_first_goal']  # Define a coluna 'possible_first_goal' como alvo
X = df.drop('possible_first_goal', axis=1) # Ele coloca no "x", então, todas as variáveis que não são a coluna 'possible_first_goal'

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

- **Separação em Features (`X`) e Target (`y`)**:
   - **`y` (target/alvo)**: A variável `y` é criada a partir da coluna `'possible_first_goal'`. Esta coluna é a variável dependente, ou seja, o que quer-se prever. Ela indica se um jogador tem ou não a possibilidade de ter marcado o primeiro gol da partida.
   - **`X` (features/características)**: A variável `X` contém todas as outras colunas do DataFrame `df`. Essas colunas são as variáveis independentes que serão usadas para treinar o modelo a prever o valor de `y`.

- **Divisão em Conjuntos de Treino e Teste**:
   - **`train_test_split()`**: Esta função, proveniente do módulo `sklearn.model_selection`, divide os dados em dois conjuntos: um para treino (`X_train`, `y_train`) e outro para teste (`X_test`, `y_test`).
   - **`test_size=0.3`**: Especifica que 30% dos dados serão reservados para o conjunto de teste, enquanto 70% serão usados para treinar o modelo. A divisão entre treino e teste é importante para avaliar o desempenho do modelo em dados que ele ainda não viu.

In [ ]:
# Utiliza o modelo de random forest pra testar as colunas mais importantes do csv escolhido
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

- **Importação do `RandomForestClassifier`**: um algoritmo de machine learning que cria múltiplas árvores de decisão durante o treinamento e usa a média dos resultados das árvores para fazer previsões.
   - A instância do `RandomForestClassifier` é criada com o parâmetro `random_state=42`, que garante que os resultados sejam reproduzíveis, ou seja, que o modelo se comporte da mesma forma a cada vez que for executado com os mesmos dados e parâmetros.
   - O Random Forest é particularmente útil para identificar as colunas (features) mais importantes, pois ele naturalmente calcula a importância de cada feature com base em como ela contribui para a redução da impureza em cada árvore.

In [ ]:
# Obtém a importância de cada feature no modelo treinado
feature_importances = model.feature_importances_
features = X.columns

# Gera uma relação entre as features (colunas) e sua importância/impacto no resultado da "'possible_first_goal'"
importance_df = pd.DataFrame({'Feature': features, 'Importance': feature_importances})
importance_df = importance_df.sort_values(by='Importance', ascending=False)
importance_df

- O atributo `model.feature_importances_` do `RandomForestClassifier` retorna um array com a importância relativa de cada feature usada no treinamento.
- Para relacionar as features com suas importâncias:
   - `features = X.columns`: Obtém os nomes das colunas do DataFrame `X`, que correspondem às features usadas no treinamento do modelo.
   - `importance_df = pd.DataFrame({'Feature': features, 'Importance': feature_importances})`: Um DataFrame é criado para relacionar cada feature com sua importância calculada pelo modelo. Este DataFrame tem duas colunas:
     - **`Feature`**: O nome da feature.
     - **`Importance`**: O valor da importância relativa dessa feature no modelo.
- `importance_df = importance_df.sort_values(by='Importance', ascending=False)`: O DataFrame é ordenado pela coluna `Importance`, em ordem decrescente. Isso significa que as features mais importantes (aquelas que mais contribuíram para as previsões do modelo) aparecerão no topo da lista.
- **`importance_df`**: O DataFrame final mostra as features ordenadas de acordo com sua importância no modelo. Isso pode ser utilizado para entender quais variáveis têm maior impacto na previsão de `'possible_first_goal'`.

In [ ]:
# Gera um gráfico para as features em ordem de importância
plt.figure(figsize=(12, 8))
plt.barh(importance_df['Feature'], importance_df['Importance'], color='skyblue')
plt.xlabel('Importancia', fontsize=14)
plt.ylabel('Features', fontsize=14)
plt.title('Features por Importancia', fontsize=16)
plt.gca().invert_yaxis()
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.tight_layout()
plt.show()

- Criação do Gráfico:
   - `plt.figure(figsize=(12, 8))`: Cria uma nova figura para o gráfico.
   - `plt.barh(top_features['Feature'], top_features['Importance'], color='skyblue')`: Plota um gráfico de barras horizontal onde:
     - O eixo Y contém os nomes das features (`top_features['Feature']`).
     - O eixo X contém as importâncias dessas features (`top_features['Importance']`).
     - A cor das barras é definida como `skyblue`.
   - `plt.xlabel('Importance', fontsize=14)` e `plt.ylabel('Feature', fontsize=14)`: Definem os rótulos dos eixos X e Y.
   - `plt.title(f'Top {top_n} Feature Importances', fontsize=16)`: Adiciona um título ao gráfico, indicando que ele mostra as 16 features mais importantes.
   - `plt.gca().invert_yaxis()`: Inverte o eixo Y para que as features mais importantes apareçam no topo.
   - `plt.xticks(fontsize=12)` e `plt.yticks(fontsize=12)`: Ajusta o tamanho da fonte dos ticks dos eixos X e Y para 12.
   - `plt.tight_layout()`: Ajusta automaticamente o layout para garantir que todos os elementos do gráfico sejam exibidos corretamente e sem sobreposição.
   - `plt.show()`: Exibe o gráfico.
   
&nbsp;&nbsp;&nbsp;&nbsp;Dessa forma, o código gera um gráfico de barras horizontal que exibe as features em ordem de importância, sendo que cada barra representa a importância de uma feature específica na previsão do modelo. Assim, esse gráfico é útil para entender quais variáveis têm maior impacto no desempenho do modelo, facilitando a interpretação.

&nbsp;&nbsp;&nbsp;&nbsp;Portanto, conclui-se, a partir do gráfico que a feature "opponent_encoded", qual o time adversário, é a que apresentou maior importância dentro do modelo, seguida pela "min_per_goal_overall", que representa o tempo médio que o jogador leva para marcar um gol.

## Hipóteses
&nbsp;&nbsp;&nbsp;&nbsp;As hipóteses formuladas se baseiam na análise exploratória dos dados e buscam entender a relação entre determinadas variáveis de desempenho dos jogadores e a probabilidade de eles marcarem o primeiro gol em uma partida:

- **Quanto menor a quantidade média de minutos por gol, maior a probabilidade de marcar o primeiro gol**: A quantidade média de minutos por gol, representada pela feature "min_per_goal_overall", indica a eficiência de um jogador em marcar gols. Ela é calculada dividindo o número total de minutos jogados pelo número de gols marcados por esse jogador. Se um jogador tem uma baixa média de minutos por gol, isso significa que ele precisa de menos tempo em campo para marcar; esse tipo de jogador é, portanto, mais eficiente. Assim, a hipótese sugere que jogadores com uma menor média de minutos por gol são mais propensos a marcar o primeiro gol em uma partida, porque tendem a levar menos tempo para marcar.

- **Quanto maior a quantidade de gols marcados por um jogador, maior a probabilidade dele marcar o primeiro gol**: A feature "goals_overall", que representa a quantidade total de gols marcados por um jogador, reflete sua capacidade geral desse jogador marcar. Dessa forma, jogadores que marcam muitos gols geralmente têm habilidades de finalização, são bem posicionados em campo e têm um papel central no ataque de sua equipe. Nesse sentido, a hipótese sugere que, por serem mais frequentemente envolvidos em situações de gol, esses jogadores têm uma maior probabilidade de marcar o primeiro gol em uma partida.

&nbsp;&nbsp;&nbsp;&nbsp;Essas hipóteses visam estabelecer correlações que, se comprovadas, podem ajudar a prever quais jogadores têm mais chances de marcar o primeiro gol em uma partida.